## 1 - Importando as bibliotecas e lendo o arquivo CSV

In [82]:
#Importando as bibliotecas
import pandas as pd 
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt

In [83]:
import plotly.io as pio
pio.renderers.default = 'browser'

In [84]:
#Lendo o arquivo CSV
df = pd.read_csv('../vehicles.csv')

## 2 - Visualizando os dados

In [85]:
#Imprimindo informações do df
df.info

<bound method DataFrame.info of        price  model_year           model  condition  cylinders fuel  odometer  \
0       9400      2011.0          bmw x5       good        6.0  gas  145000.0   
1      25500         NaN      ford f-150       good        6.0  gas   88705.0   
2       5500      2013.0  hyundai sonata   like new        4.0  gas  110000.0   
3       1500      2003.0      ford f-150       fair        8.0  gas       NaN   
4      14900      2017.0    chrysler 200  excellent        4.0  gas   80903.0   
...      ...         ...             ...        ...        ...  ...       ...   
51520   9249      2013.0   nissan maxima   like new        6.0  gas   88136.0   
51521   2700      2002.0     honda civic    salvage        4.0  gas  181500.0   
51522   3950      2009.0  hyundai sonata  excellent        4.0  gas  128000.0   
51523   7455      2013.0  toyota corolla       good        4.0  gas  139573.0   
51524   6300      2014.0   nissan altima       good        4.0  gas       NaN

In [86]:
#Visualizando os tipos de cada coluna
df.dtypes

price             int64
model_year      float64
model            object
condition        object
cylinders       float64
fuel             object
odometer        float64
transmission     object
type             object
paint_color      object
is_4wd          float64
date_posted      object
days_listed       int64
dtype: object

## 3 - Preparação dos dados
- Convertendo os tipos dos dados
- Tratando ausentes
- Identificando dados duplicados
- Enriquecendo os dados

In [87]:
#Convertendo as colunas para os tipos corretos
df['model_year'] = df['model_year'].astype('Int64')
df['cylinders'] = df['cylinders'].astype('Int64')
df['odometer'] = df['odometer'].astype('Int64')
df['is_4wd'] = df['is_4wd'].fillna(0).astype(bool)
df['date_posted'] = pd.to_datetime(df['date_posted'])


#Checando a alteração
df.dtypes

price                    int64
model_year               Int64
model                   object
condition               object
cylinders                Int64
fuel                    object
odometer                 Int64
transmission            object
type                    object
paint_color             object
is_4wd                    bool
date_posted     datetime64[ns]
days_listed              int64
dtype: object

In [88]:
#Procurando por valores ausentes
df_nan = df.isna().sum()
print(df_nan)

price              0
model_year      3619
model              0
condition          0
cylinders       5260
fuel               0
odometer        7892
transmission       0
type               0
paint_color     9267
is_4wd             0
date_posted        0
days_listed        0
dtype: int64


In [89]:
#Tratando valores ausentes nas colunas model_year e paint_color
df['paint_color'] = df['paint_color'].fillna('unknown')

for model in df['model'].unique():
    most_common_year = df.loc[(df['model'] == model) & (df['model_year'].notna()), 'model_year'].mode()

    if not most_common_year.empty:
        df.loc[(df['model'] == model) & (df['model_year'].isna()), 'model_year'] = most_common_year.iloc[0]


In [90]:
#Checando as alterações
print(df.isna().sum())

price              0
model_year         0
model              0
condition          0
cylinders       5260
fuel               0
odometer        7892
transmission       0
type               0
paint_color        0
is_4wd             0
date_posted        0
days_listed        0
dtype: int64


In [91]:
#Procurando por valores duplicados
df_duplicated = df.duplicated().sum()
print(df_duplicated)

0


In [92]:
#Criando a coluna vehicle_age (idade aproximada do carro)
df['vehicle_age'] = df['date_posted'].dt.year - df['model_year']

print(df['vehicle_age'])

0         7
1         5
2         6
3        16
4         2
         ..
51520     5
51521    16
51522     9
51523     5
51524     4
Name: vehicle_age, Length: 51525, dtype: Int64


## 4 - Análise exploratória de dados

In [93]:
#Criando um gráfico para analisar a distribuição dos preços dos carros

df_filtered = df[df['price'] < 70000] #Filtrando por preços abaixo de 70.000 (para melhor visualização)


fig = px.histogram(
    df_filtered,
    x='price',
    nbins=50,
    title='Distribuição dos preços dos carros (até $70.000)',
    labels={'price': 'Preço ($)'},
    color_discrete_sequence=['skyblue']
)

fig.update_layout(
    bargap=0.1,
    xaxis_title='Preço ($)',
    yaxis_title='Quantidade de carros',
    template='simple_white'
)

fig.show()

In [94]:
#Criando um gráfico de dispersão para analisar a relação entre preço e quilometragem
fig = px.scatter(
    df,
    x='odometer',
    y='price',
    title='Relação entre preço e quilometragem dos carros',
    labels={'odometer': 'Quilometragem', 'price': 'Preço ($)'},
    opacity=0.6,
)

fig.show()

In [ ]:
#Mostrando a relação entre a média de preço e a idade do carro

price_by_age = df.groupby('vehicle_age')['price'].mean().reset_index()

fig = px.line(
    price_by_age,
    x='vehicle_age',
    y='price',
    title='Preço Médio por Idade do Carro',
    labels={'vehicle_age': 'Idade do carro (anos)', 'price': 'Preço Médio (R$)'},
)

fig.show()

In [100]:
#Analisando a relação entre o preço médio e a marca do carro

#Criando a coluna brand a partir da primeira palavra de 'model'
df['brand'] = df['model'].str.split().str[0]

#Calculando o preço médio por marca
price_by_brand = df.groupby('brand')['price'].mean().reset_index().sort_values(by='price', ascending=False)

#Criando o gráfico de barras
fig = px.bar(
    price_by_brand,
    x='brand',
    y='price',
    title='Preço médio por marca de carro',
    labels={'brand': 'Marca', 'price': 'Preço médio ($)'},
    color_discrete_sequence=['steelblue']
)

fig.update_layout(xaxis_tickangle=-45)
fig.show()

In [101]:
#Calculando o tempo médio de venda por marca

days_by_brand = df.groupby('brand')['days_listed'].mean().reset_index().sort_values(by='days_listed', ascending=False)

#Criando o gráfico de barras
fig = px.bar(
    days_by_brand,
    x='brand',
    y='days_listed',
    title='Tempo médio de venda por marca de carro',
    labels={'brand': 'Marca', 'days_listed': 'Dias listados'},
    color_discrete_sequence=['indianred']
)

fig.update_layout(xaxis_tickangle=-45)
fig.show()

In [108]:
#Analisando o tempo médio de venda (dias) por faixa de preço

bins = [0, 5000, 10000, 15000, 20000, 30000, 40000, 50000, 70000, 100000, df['price'].max()]
labels = [
    '≤5k', '5–10k', '10–15k', '15–20k',
    '20–30k', '30–40k', '40–50k', '50–70k',
    '70–100k', '100k+'
]
df['price_range'] = pd.cut(df['price'], bins=bins, labels=labels, include_lowest=True)

#Calculando tempo médio de venda por faixa de preço
avg_days = df.groupby('price_range', observed=False)['days_listed'].mean().reset_index()

#Criando o gráfico de barras
fig = px.bar(
    avg_days,
    x='price_range',
    y='days_listed',
    title='Tempo médio de venda por faixa de preço',
    labels={'price_range': 'Faixa de preço ($)', 'days_listed': 'Dias listados'},
    color_discrete_sequence=['mediumseagreen']
)

fig.update_layout(xaxis_tickangle=-45)
fig.show()

In [ ]:
#Analisando o tempo de venda em relação ao tipo do carro

#Agrupando por tipo e calculando o tempo médio de venda
avg_days_by_type = df.groupby('type')['days_listed'].mean().reset_index().sort_values(by='days_listed', ascending=False)

#Criando o gráfico de barras
fig = px.bar(
    avg_days_by_type,
    x='type',
    y='days_listed',
    title='Tempo médio de venda por tipo de carro',
    labels={'type': 'Tipo de carro', 'days_listed': 'Dias listados'},
    color_discrete_sequence=['teal']
)

fig.update_layout(xaxis_tickangle=-45)
fig.show()